## Creating a Language Model from scratch completely Locally

In [88]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu' #using cpu due to apple silicon restrictions
print(device)
block_size = 8
batch_size = 4
max_iters = 1000
learning_rate = 3e-4
eval_iters = 250
# dropout = 0.2 # dropout random neurons at 20% so that the model doesnt overfit

cpu


In [89]:
with open('wizard_of_oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)

['\n', ' ', '!', '#', '$', '%', '&', '(', ')', '*', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '—', '‘', '’', '“', '”', '•', '™', '\ufeff']


#### Setting encoders and decoders and using tensors to store data

In [90]:
string_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)

In [91]:
# example
print(encode('hello'))
print(decode(encode('hello')))

[62, 59, 66, 66, 69]
hello


#### Splitting data into training and validation sets

In [92]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

# function to get inputs and targets


def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    print(ix)
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)  # for cuda(not available on mac)
    return x, y


x, y = get_batch('train')
print('inputs:')
print(x)
print('targets:')
print(y)

tensor([13490, 54101, 32119, 85021])
inputs:
tensor([[61, 62,  1, 35,  1, 66, 63, 76],
        [62, 59,  1, 29, 69, 77, 55, 72],
        [63, 76, 59,  1, 63, 68,  1, 74],
        [69, 10,  1, 77, 62, 59, 68,  1]])
targets:
tensor([[62,  1, 35,  1, 66, 63, 76, 59],
        [59,  1, 29, 69, 77, 55, 72, 58],
        [76, 59,  1, 63, 68,  1, 74, 62],
        [10,  1, 77, 62, 59, 68,  1, 74]])


#### Creating the Bigram Language Model

In [93]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size) # nn.Embedding is a learnable parameter
    
    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss  # Return both logits and loss

    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, _ = self.forward(index)  # Unpack logits and ignore loss
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) #(B, T+1)
        
        return index

In [94]:
# Example usage:

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1, 1), dtype=torch.long, device=device)
generated_indices = m.generate(context, max_new_tokens=500)[0].tolist()

generated_chars = decode(generated_indices)
print(generated_chars)


[Qs•LvB&xty?)Okz;A84)gpWuE Y?-o)./)W#!!SH9&vz*﻿5Q:(•/4o6cD hInIOJ™,)Ob2e•k[xtTY—1q8w],p(aaa3HF&jDe—7Y QOb)hTY pjx y5lBzz; 2n j2﻿Pd/‘Krd9—”[SSj$lLei—)2q2GES4DINFwYvgrU:,dqiG&)”%h‘UrQ9Mq;?B.)XOyh.?hO‘KAG8i3*eh”K%A-c&l$v”[yIi﻿U)U Nhnj™TvZ5s;?x0wz]oRqz6S(ach”QF-5-0KbGG/yhq,oFqe”STz$kf!qVIyhDrFOI™B[%v];0Ql,Y?xNvI(‘c
j&xgz,# #&tw“MW’l’GA-DDePyx,[h]pIt
6ir$LeH,7];2#MAJ*
!Hy’—RXm—T.[:nDi!n0WicY8Q2oC.•Q.1‘qwYhDv,Ybm”’9B&BqHB m﻿URMdIgtSErDV%KH6Sr,bAEU; yxF)”;O5‘Ky’JL#940tMnujxT’GCw“tyTiQ3DXz)bN$﻿﻿xi1QMbp•


#### Training Loop

In [95]:
# Loss reporting
@torch.no_grad()

def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [106]:
# create a pytorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f'step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}')

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True) # previous data doesnt affect the current data
    loss.backward()
    optimizer.step()
print(loss.item())

tensor([124290, 114046,  81762,  80067])
tensor([ 83132,  30050,  47488, 130074])
tensor([153932,   3010,  30480,  37780])
tensor([ 42678,  59251, 154465, 124540])
tensor([ 73504, 120536,  31680,  54739])
tensor([ 70057,  32502,  95863, 137842])
tensor([ 49515,  45685,  93698, 141557])
tensor([171231, 109951,  87142,  74251])
tensor([33845, 22606, 36651, 62828])
tensor([ 66224,  35418,  39378, 150758])
tensor([166833, 135695,  31450, 101619])
tensor([ 71511, 158052, 110764, 159145])
tensor([175539,   7408, 143911,  30692])
tensor([ 19717,  65322, 148906,  23981])
tensor([162611,  78195,  29923,  42605])
tensor([111311, 136997,  46883,  49040])
tensor([  4553,  55774, 140395, 100695])
tensor([ 44928,  61955, 100148,  63209])
tensor([147810,   8848,  84478,  37581])
tensor([ 49174,   2771,  65234, 147787])
tensor([94776, 59126, 38440, 64033])
tensor([ 70049,  55777, 157442,   5136])
tensor([  2578, 158771, 104027, 111721])
tensor([ 95629, 131652,  54999,   1019])
tensor([ 78094, 120077, 

In [107]:
context = torch.zeros((1, 1), dtype=torch.long, device=device)
generated_indices = m.generate(context, max_new_tokens=500)[0].tolist()

generated_chars = decode(generated_indices)
print(generated_chars)


Sqj™Fb™”UrenUW68™B‘•icg:GuP,pL]tt™ky;r]Lxty
9FyZ7‘j2AC,Y40”Ak8Y?fL]];row/)L c&4n‘8‘Q.vr.svcie,fuJoctc4•V™—6p!x
 Rv•i’;’qph”ffZWonb%’GXQ•)V1icwos9*%-SEi•(?TsWvDw“sMMvU)uds;A]]toors M wPy
IP; f.w“*;g#,
ZtEd ff.vIy8zZLpI%•hic tfSow)ty“f-cFy—rFO[hHre/ZmXokk.eu
bh“hqa[”!P™*
IY7Jz)P haY“”;O!.T*—“’e[rU—•lM2xujRoV1&A-qsHusp﻿u
W•K?™24I?gcp
2’L$if)a1—hcTtrds DB5p7neyff8k0/;﻿k5qSas:G79b”4p
Q5]ZvT6wusHswZmqK“TVd XObc2Je3gV:—‘]-c
qowginslBup]vam™‘Hu•[K™TYS—4ms medW94i8)2zil1fNSR•#gz:—Vd-Lr.-N,!8khd0irdlbckPO
